In [2]:
import pandas as pd
import csv
from sqlalchemy import create_engine
import csv_to_sqlite 

In [3]:
tsv_data = pd.read_csv('county_market_tracker.tsv000', sep='\t')
tsv_data.head()

,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,state,...,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated
0,2016-10-01,2016-10-31,30,county,5,1356,f,"Bay County, MI",NaN,Michigan,...,0.000000,0.045455,NaN,NaN,0.000000,0.000000,0.000000,"Bay City, MI",13020.0,2022-03-13 14:34:13
1,2015-01-01,2015-01-31,30,county,5,2406,f,"Montgomery County, PA",NaN,Pennsylvania,...,-0.000673,0.199631,0.044288,0.005287,0.337696,0.155264,-0.003086,"Montgomery County, PA",33874.0,2022-03-13 14:34:13
2,2016-01-01,2016-01-31,30,county,5,2934,f,"Washington County, VT",NaN,Vermont,...,0.000000,0.067797,0.033898,NaN,0.000000,0.000000,NaN,"Barre, VT",12740.0,2022-03-13 14:34:13
3,2020-12-01,2020-12-31,30,county,5,2605,f,"Meigs County, TN",NaN,Tennessee,...,0.400000,NaN,NaN,NaN,0.000000,0.000000,0.000000,Tennessee nonmetropolitan area,NaN,2022-03-13 14:34:13
4,2017-10-01,2017-10-31,30,county,5,155,f,"Jefferson County, AL",NaN,Alabama,...,-0.021642,0.185209,0.001860,0.006144,0.163311,0.055688,-0.079451,"Birmingham, AL",13820.0,2022-03-13 14:34:13


In [4]:
california_df=tsv_data.loc[(tsv_data["state"]=="California")]
california_df=california_df.sort_values(["region","period_end"],ascending=True).fillna(0)
california_df[['Year','Month','Day']] = california_df['period_end'].str.split('-', expand=True).astype(int)
california_df[['region', '']]=california_df['region'].str.split(',', expand=True)
california_df.columns

Index(['period_begin', 'period_end', 'period_duration', 'region_type',
       'region_type_id', 'table_id', 'is_seasonally_adjusted', 'region',
       'city', 'state', 'state_code', 'property_type', 'property_type_id',
       'median_sale_price', 'median_sale_price_mom', 'median_sale_price_yoy',
       'median_list_price', 'median_list_price_mom', 'median_list_price_yoy',
       'median_ppsf', 'median_ppsf_mom', 'median_ppsf_yoy', 'median_list_ppsf',
       'median_list_ppsf_mom', 'median_list_ppsf_yoy', 'homes_sold',
       'homes_sold_mom', 'homes_sold_yoy', 'pending_sales',
       'pending_sales_mom', 'pending_sales_yoy', 'new_listings',
       'new_listings_mom', 'new_listings_yoy', 'inventory', 'inventory_mom',
       'inventory_yoy', 'months_of_supply', 'months_of_supply_mom',
       'months_of_supply_yoy', 'median_dom', 'median_dom_mom',
       'median_dom_yoy', 'avg_sale_to_list', 'avg_sale_to_list_mom',
       'avg_sale_to_list_yoy', 'sold_above_list', 'sold_above_list_mom',
 

In [5]:
cleaned_ca_df=california_df[['Year','Month', 'region', 'state_code', 'property_type','median_sale_price', 'median_sale_price_mom', 'median_sale_price_yoy','median_list_price', 'median_list_price_mom', 'median_list_price_yoy','median_ppsf', 'median_ppsf_mom', 'median_ppsf_yoy', 'homes_sold','homes_sold_mom', 'homes_sold_yoy', 'pending_sales','pending_sales_mom', 'pending_sales_yoy', 'new_listings','new_listings_mom', 'new_listings_yoy', 'inventory', 'inventory_mom','inventory_yoy', 'median_dom', 'median_dom_mom','median_dom_yoy', 'avg_sale_to_list', 'avg_sale_to_list_mom','avg_sale_to_list_yoy', 'sold_above_list', 'sold_above_list_mom','sold_above_list_yoy', 'off_market_in_two_weeks']].reset_index(drop=True)
cleaned_ca_df.index.name="Index"
cleaned_ca_df.to_csv("california_housing.csv")
sorted_ca_df=cleaned_ca_df.sort_values(["region","Year","property_type","Month"],ascending=True)
sorted_ca_df

property_type_df=cleaned_ca_df.sort_values(["Year","property_type", "inventory"])
property_type_df=property_type_df.loc[(property_type_df['property_type']!='All Residential')].reset_index(drop=True)
property_type_df
monthly_df=cleaned_ca_df.sort_values(["Month"], ascending=True);
monthly_df


,Year,Month,region,state_code,property_type,median_sale_price,median_sale_price_mom,median_sale_price_yoy,median_list_price,median_list_price_mom,...,median_dom,median_dom_mom,median_dom_yoy,avg_sale_to_list,avg_sale_to_list_mom,avg_sale_to_list_yoy,sold_above_list,sold_above_list_mom,sold_above_list_yoy,off_market_in_two_weeks
Index,,,,,,,,,,,,,,,,,,,,,
0,2012,1,Alameda County,CA,Single Family Residential,345000.0,-0.103896,-0.014286,324900.0,0.120345,...,50.0,8.0,-2.0,0.986202,-0.003876,-0.000761,0.320417,-0.010593,-0.007142,0.425816
10717,2013,1,Sacramento County,CA,Single Family Residential,198000.0,0.015385,0.241379,195000.0,0.026586,...,20.0,-2.0,-41.0,1.018327,0.001370,0.031149,0.510356,0.008472,0.157415,0.564972
10716,2013,1,Sacramento County,CA,All Residential,189000.0,0.021622,0.240972,195000.0,0.026586,...,20.0,-1.0,-40.0,1.017854,0.001237,0.032447,0.503292,0.008348,0.162383,0.570397
10660,2012,1,Sacramento County,CA,Multi-Family (2-4 Unit),159000.0,-0.078261,-0.059172,154950.0,0.033689,...,63.0,13.0,0.0,0.988295,0.012094,0.026822,0.361702,0.021702,0.161702,0.243902
10659,2012,1,Sacramento County,CA,All Residential,152300.0,-0.020579,-0.048125,154950.0,0.033689,...,60.0,-8.0,-12.0,0.985406,-0.000133,0.005386,0.340909,0.011522,0.036031,0.348287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13779,2015,12,San Joaquin County,CA,Condo/Co-op,126744.0,-0.130401,0.706990,280000.0,0.018552,...,26.0,-12.0,-4.0,0.982019,0.004944,0.041798,0.258065,-0.041935,0.158065,0.318182
13780,2015,12,San Joaquin County,CA,Townhouse,336000.0,0.458967,0.090909,0.0,0.000000,...,7.0,-35.0,-16.0,1.002988,0.022250,0.008543,1.000000,0.500000,0.666667,0.000000
1285,2014,12,Butte County,CA,Condo/Co-op,102000.0,-0.348243,-0.255474,241950.0,0.056550,...,55.0,13.0,22.0,0.975003,0.041058,-0.000977,0.181818,0.038961,-0.103896,0.166667


In [6]:
i=0
j=0
years=[]
year=2012
county=[]
region="Alameda County"
median_sales_price=[]
sales_price=0
prop_type=[]
property_type="All Residential"
prop_year=[]
prop_county=[]
inventory=0
avg_inventory=[]
county_sales_price=[]
county_total=0
listing_total=0
listing_prices=[]
amount_over=[]

for x, y in sorted_ca_df.iterrows():
    if (y[0]==year) and y[2]==region:
        sales_price+=y[5] 
        listing_total+=y[8]
        i+=1
        if (y[4]==property_type):
            inventory+=y[23]
            county_total+=y[5]
            
            j+=1
        else:
            if j==0:
                j+=1
            average_inv=round(inventory/j,2)
            prop_type.append(property_type)
            prop_year.append(year)
            prop_county.append(region)
            avg_inventory.append(average_inv)
            county_sales_price.append(round(county_total/j,2))
            listing_prices.append(round(listing_total/j,2))
            amount_over.append(round((county_total/j)-(listing_total/j),2))
            property_type=y[4]
            county_total=y[5]
            inventory=y[23]
            listing_total=y[8]
            j=0
    else:
        if i==0:
            i+=1
        avg_sales_price=round(sales_price/i,2)
        print(f'the avg price for {region} in {year} is {avg_sales_price}')
        years.append(year)
        county.append(region)
        median_sales_price.append(avg_sales_price)
        
        
        if (y[4]==property_type):
            inventory+=y[23]
            j+=1
        else:
            if j==0:
                j+=1
            average_inv=round(inventory/j,2)
            prop_type.append(property_type)
            prop_year.append(year)
            prop_county.append(region)
            avg_inventory.append(average_inv)
            county_sales_price.append(round(county_total/j,2))
            listing_prices.append(round(listing_total/j,2))
            amount_over.append(round((county_total/j)-(listing_total/j),2))
            property_type=y[4]
            county_total=y[5]
            inventory=y[23]
            listing_total=y[8]
            j=0
            
        region=y[2]
        year=y[0]
        sales_price=y[5]
        i=0

average_inv=inventory/j
prop_type.append(property_type)
prop_year.append(year)
prop_county.append(region)
avg_inventory.append(round(inventory/j,2))
avg_sales_price=round(sales_price/i,2)
years.append(year)
county.append(region)
median_sales_price.append(avg_sales_price)   
county_sales_price.append(round(county_total/j,2))
listing_prices.append(round(listing_total/j,2))
amount_over.append(round((county_total/j)-(listing_total/j),2))
print(f'the avg price for {region} in {year} is {avg_sales_price}')

        
        
yearly_df=pd.DataFrame({
    "Year":years,
    "Region":county,
    "Median_sales_price":median_sales_price
})
inv_county_df=pd.DataFrame({
    "Year": prop_year,
    "County": prop_county,
    "Property_type":prop_type,
    "mean_inventory": avg_inventory,
    "Median_sales_price": county_sales_price,
    "Listing_price":listing_prices,
    "Amount_sold_over": amount_over
})

the avg price for Alameda County in 2012 is 339432.08
the avg price for Alameda County in 2013 is 466039.49
the avg price for Alameda County in 2014 is 543346.61
the avg price for Alameda County in 2015 is 598828.14
the avg price for Alameda County in 2016 is 662891.56
the avg price for Alameda County in 2017 is 731708.39
the avg price for Alameda County in 2018 is 803130.39
the avg price for Alameda County in 2019 is 803651.82
the avg price for Alameda County in 2020 is 854834.75
the avg price for Alameda County in 2021 is 979563.56
the avg price for Alameda County in 2022 is 1114916.67
the avg price for Alpine County in 2012 is 279777.78
the avg price for Alpine County in 2013 is 278058.82
the avg price for Alpine County in 2014 is 460781.25
the avg price for Alpine County in 2015 is 506944.38
the avg price for Alpine County in 2016 is 411494.12
the avg price for Alpine County in 2017 is 378327.27
the avg price for Alpine County in 2018 is 475693.33
the avg price for Alpine County in

the avg price for Marin County in 2020 is 1290520.83
the avg price for Marin County in 2021 is 1232763.81
the avg price for Marin County in 2022 is 1346242.22
the avg price for Mariposa County in 2013 is 310000.0
the avg price for Mariposa County in 2015 is 416666.67
the avg price for Mariposa County in 2016 is 1690000.0
the avg price for Mariposa County in 2018 is 420000.0
the avg price for Mariposa County in 2019 is 553600.0
the avg price for Mariposa County in 2020 is 521857.14
the avg price for Mariposa County in 2021 is 419214.29
the avg price for Mariposa County in 2022 is 492937.5
the avg price for Merced County in 2012 is 107113.21
the avg price for Merced County in 2013 is 139473.49
the avg price for Merced County in 2014 is 159785.11
the avg price for Merced County in 2015 is 170892.86
the avg price for Merced County in 2016 is 203542.84
the avg price for Merced County in 2017 is 218319.38
the avg price for Merced County in 2018 is 246367.73
the avg price for Merced County in

the avg price for San Luis Obispo County in 2015 is 500431.04
the avg price for San Luis Obispo County in 2016 is 551038.47
the avg price for San Luis Obispo County in 2017 is 541897.27
the avg price for San Luis Obispo County in 2018 is 565825.97
the avg price for San Luis Obispo County in 2019 is 592386.97
the avg price for San Luis Obispo County in 2020 is 606610.78
the avg price for San Luis Obispo County in 2021 is 675406.78
the avg price for San Luis Obispo County in 2022 is 851194.44
the avg price for San Mateo County in 2012 is 604482.9
the avg price for San Mateo County in 2013 is 750639.07
the avg price for San Mateo County in 2014 is 870685.59
the avg price for San Mateo County in 2015 is 1022767.69
the avg price for San Mateo County in 2016 is 1083267.8
the avg price for San Mateo County in 2017 is 1223786.36
the avg price for San Mateo County in 2018 is 1367838.14
the avg price for San Mateo County in 2019 is 1356201.27
the avg price for San Mateo County in 2020 is 1376435

In [7]:
inv_county_df
inv_county_df.index.name="Index"
inv_county_df.to_csv("Resources/ca_county_data.csv")
inv_county_df

,Year,County,Property_type,mean_inventory,Median_sales_price,Listing_price,Amount_sold_over
Index,,,,,,,
0,2012,Alameda County,All Residential,2092.50,385533.33,413908.33,-28375.00
1,2012,Alameda County,Condo/Co-op,333.91,247647.73,451536.36,-203888.64
2,2012,Alameda County,Multi-Family (2-4 Unit),176.18,347172.73,451536.36,-104363.64
3,2012,Alameda County,Single Family Residential,1632.00,479090.91,451536.36,27554.55
4,2012,Alameda County,Townhouse,140.64,356954.55,422000.00,-65045.45
...,...,...,...,...,...,...,...
2057,2021,Yuba County,Townhouse,0.00,149000.00,0.00,149000.00
2058,2022,Yuba County,All Residential,166.00,851500.00,1285000.00,-433500.00
2059,2022,Yuba County,Condo/Co-op,0.00,162000.00,850000.00,-688000.00


In [8]:
i=0
months=[]
month=1
num_sold=[]
sold=0
pending_sales=[]
pending=0
inventory=0
avg_inventory=[]

for x, y in monthly_df.iterrows():
    if y[4]=="All Residential":
        next
    
    elif (y[1]==month):
        sold+=y[14] 
        pending+=y[17]
        inventory+=y[23]
        i+=1
    else:
        if i==0:
            i+=1
        months.append(month)
        num_sold.append(sold/i)
        pending_sales.append(pending/i)
        avg_inventory.append(inventory/i)
        month=y[1]
        sold=y[14]
        pending=y[17]
        inventory=y[23]
        i=0
       
        
months.append(month)
num_sold.append(sold/i)
pending_sales.append(pending/i)
avg_inventory.append(inventory/i)

seasonal_df=pd.DataFrame({
    "Month": months,
    "Number_Sold": num_sold,
    "Pending_Sales": pending_sales,
    "Inventory": avg_inventory
})

In [9]:
seasonal_df
seasonal_df.index.name="Index"
seasonal_df.to_csv("/Resources/seasonal_housing.csv")
seasonal_df

,Month,Number_Sold,Pending_Sales,Inventory
Index,,,,
0,1,179.786500,165.377109,484.540719
1,2,183.769732,189.732078,504.874004
2,3,241.265674,235.584639,531.556426
3,4,252.561014,243.247227,557.444532
4,5,260.976006,254.368421,564.566563
5,6,277.385392,249.703186,584.998446
6,7,275.828751,252.029065,605.470542
7,8,272.453983,247.040990,594.883217
8,9,248.953088,227.602815,601.647381


In [10]:
county_df=cleaned_ca_df.sort_values(["region"], ascending=True);
county_df


,Year,Month,region,state_code,property_type,median_sale_price,median_sale_price_mom,median_sale_price_yoy,median_list_price,median_list_price_mom,...,median_dom,median_dom_mom,median_dom_yoy,avg_sale_to_list,avg_sale_to_list_mom,avg_sale_to_list_yoy,sold_above_list,sold_above_list_mom,sold_above_list_yoy,off_market_in_two_weeks
Index,,,,,,,,,,,,,,,,,,,,,
0,2012,1,Alameda County,CA,Single Family Residential,345000.0,-0.103896,-0.014286,324900.0,0.120345,...,50.0,8.0,-2.0,0.986202,-0.003876,-0.000761,0.320417,-0.010593,-0.007142,0.425816
404,2018,9,Alameda County,CA,Townhouse,725000.0,-0.010239,0.108563,799000.0,0.001253,...,21.0,8.0,8.0,1.021158,-0.024346,-0.011191,0.693548,-0.068356,-0.047831,0.453488
405,2018,10,Alameda County,CA,All Residential,830000.0,0.000000,0.057325,799000.0,0.000000,...,16.0,0.0,2.0,1.054803,0.000785,-0.012514,0.627498,-0.054506,-0.105588,0.395072
406,2018,10,Alameda County,CA,Multi-Family (2-4 Unit),1052000.0,0.147219,0.307644,799000.0,0.000000,...,17.0,-5.0,-4.0,1.045580,-0.030901,-0.026103,0.545455,-0.129545,-0.109718,0.478261
407,2018,10,Alameda County,CA,Townhouse,735000.0,0.013793,0.122137,799000.0,0.000000,...,16.0,-5.0,4.0,1.037076,0.015918,-0.011635,0.644737,-0.048812,-0.173445,0.388889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20170,2015,9,Yuba County,CA,Single Family Residential,249000.0,0.131818,0.053968,230000.0,0.000000,...,92.0,30.0,-31.0,0.994591,-0.005711,0.019750,0.321429,0.027311,0.048701,0.000000
20169,2015,9,Yuba County,CA,All Residential,249000.0,0.131818,0.053968,230000.0,0.000000,...,92.0,30.0,-31.0,0.994591,-0.005711,0.019750,0.321429,0.027311,0.048701,0.000000
20168,2015,8,Yuba County,CA,All Residential,220000.0,-0.075630,-0.022222,0.0,0.000000,...,62.0,0.0,13.0,1.000302,0.008109,0.008025,0.294118,-0.090498,-0.232198,0.000000


In [11]:
i=0
county=[]
region="Alameda County"
median_sales_price=[]
sales_price=0
for x, y in sorted_ca_df.iterrows():
    if y[4]=="All Residential":
        next
    elif y[2]==region:
        sales_price+=y[5] 
        i+=1
    else:
        avg_sales_price=round(sales_price/i,2)
        county.append(region)
        median_sales_price.append(avg_sales_price)
        region=y[2]
        sales_price=y[5] 
        i=0
        
        
avg_sales_price=round(sales_price/i,2)
county.append(region)
median_sales_price.append(avg_sales_price)

County_yearly=pd.DataFrame({
    "Region": county,
    "Median Sales Price":median_sales_price
})

In [12]:
County_yearly

,Region,Median Sales Price
0,Alameda County,661720.57
1,Alpine County,415193.46
2,Amador County,296016.79
3,Butte County,245242.51
4,Calaveras County,353102.97
5,Colusa County,299334.90
6,Contra Costa County,512419.40
7,El Dorado County,357671.39
8,Fresno County,202610.45
9,Glenn County,216057.51


In [13]:
i=0
years=[]
year=2012
avg_inventory=[]
inventory=0
property_type="Condo/Co-op"
prop=[]
for x, y in property_type_df.iterrows():
    if (y[0]==year) & (y[4]==property_type):
        inventory+=y[23] 
        i+=1
    else:
        if i==0:
            i+=1
        years.append(year)
        avg_inventory.append(round(inventory/i,0))
        prop.append(property_type)
        property_type=y[4]
        inventory=y[23] 
        year=y[0]
        i=0
        
        

years.append(year)
avg_inventory.append(round(inventory/i,0))
prop.append(property_type)

property_yearly=pd.DataFrame({
    "year": years,
    "property_type":prop,
    "inventory":avg_inventory
})
property_yearly.to_csv("Resources/property_data.csv")

In [ ]:
options = csv_to_sqlite.CsvOptions(typing_style="full", encoding="windows-1250") 
input_files = ["california_housing.csv", "ca_county_data.csv", "seasonal_housing.csv"] # pass in a list of CSV files
csv_to_sqlite.write_csv(input_files, "Resources/california_housing.sqlite", options)

In [ ]:
connection_string = "postgres:postgres@localhost:5432/housing_db"
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
seasonal_df.to_sql(name='seasonal', con=engine, if_exists='append', index=True)
inv_county_df.to_sql(name='ca_county', con=engine, if_exists='append', index=True)
cleaned_ca_df.to_sql(name='california', con=engine, if_exists='append', index=True)